## Scrapy 发送邮件详解
　　参考：[scrapy发送邮件](https://www.cnblogs.com/mangM/p/10790591.html)<br>
　　　　　[Scrapy使用MailSender发送邮件](https://developer.aliyun.com/article/810928)<br>
　　　　　[Scrapy监控爬虫状况，邮件发送功能及爬虫停止信息](https://www.jianshu.com/p/36ae2c2a016f)<br>
　　　　　[Scrapy : Stats Collection, Sending e-mail](https://blog.csdn.net/weixin_46192930/article/details/107480126)<br>
　　　　　[Scrapy学习笔记-使用signals来监控spider的状态](https://blog.csdn.net/fengxueersui/article/details/80930621)<br>
　　　　　[使用scrapy的mail模块发送邮件](https://blog.csdn.net/You_are_my_dream/article/details/60868329)<br>

In [1]:
# 首先在settings同级的目录下创建extendions(扩展)文件夹，

import logging
from scrapy import signals
from scrapy.exceptions import NotConfigured
from scrapy.mail import MailSender
logger = logging.getLogger(__name__)
class SendEmail(object):

    def __init__(self,sender,crawler):
        self.sender = sender
        crawler.signals.connect(self.spider_idle, signal=signals.spider_idle)
        crawler.signals.connect(self.spider_closed, signal=signals.spider_closed)

    @classmethod
    def from_crawler(cls,crawler):
        if not crawler.settings.getbool('MYEXT_ENABLED'):
            raise NotConfigured

        mail_host = crawler.settings.get('MAIL_HOST') # 发送邮件的服务器
        mail_port = crawler.settings.get('MAIL_PORT') # 邮件发送者
        mail_user = crawler.settings.get('MAIL_USER') # 邮件发送者
        mail_pass = crawler.settings.get('MAIL_PASS') # 发送邮箱的密码不是你注册时的密码，而是授权码！！！切记！

        sender = MailSender(mail_host,mail_user,mail_user,mail_pass,mail_port) #由于这里邮件的发送者和邮件账户是同一个就都写了mail_user了
        h = cls(sender,crawler)

        return h

    def spider_idle(self,spider):
        logger.info('idle spider %s' % spider.name)

    def spider_closed(self, spider):
        logger.info("closed spider %s", spider.name)
        body = 'spider[%s] is closed' %spider.name
        subject = '[%s] good!!!' %spider.name
        # self.sender.send(to={'zfeijun@foxmail.com'}, subject=subject, body=body)
        return self.sender.send(to={'zfeijun@foxmail.com'}, subject=subject, body=body)
    
# 这里为什么是return self.sender.send，是因为直接用sender.send会报builtins.AttributeError: 'NoneType' object has no attribute 'bio_read'的错误（邮件会发送成功），具体原因不是很懂，有大牛知道的可以指导一下。

# 解决方法:[参考](https://github.com/scrapy/scrapy/issues/3478)

#     在sender.send前加return就好了。

# 在扩展中写好代码后，需要在settings中启用

EXTENSIONS = {
    # 'scrapy.extensions.telnet.TelnetConsole': 300,
    'bukalapak.extendions.sendmail.SendEmail': 300,
}
MYEXT_ENABLED = True

ModuleNotFoundError: No module named 'scrapy'